# Relative import

This page discusses the implementation details of the relative imports in python.

## Resolving

Python resolves the relative imports using the `__package__` attribute of the importing module. If the `__package__` attribute is set to `None` or an empty string (`''`), an `ImportError: attempted relative import with no known parent package` is raised.

---

The following cells creates folder with python code.

In [3]:
#init
mkdir package

The module supposed to be imported:

In [4]:
#file package/import_me.py
print("I was imported")

The importing module:

In [13]:
#file package/load.py
print("package:", __package__)
from . import import_me

If the module is run as a script, it will have the `__package__ = None`.

In [14]:
python3 package/load.py
true

package: None
Traceback (most recent call last):
  File "/tmp/tmprh915u9z/package/load.py", line 2, in <module>
    from . import import_me
ImportError: attempted relative import with no known parent package


Python is unable to resolve the relative import.

The following cell runs imports the file as a python module, resulting in `__package__=package`.

In [15]:
python3 -m package.load

package: package
I was imported


The relative importing goes fine.

## `__main__` module

A common issue related to relative imports in Python occurs when attempting to execute a Python file that contains relative imports.

Check out [this](https://stackoverflow.com/questions/16981921/relative-imports-in-python-3) stackoverflow discussion of what actually happens.

Python uses the module's `__name__` attribute of the module when loading imports into the module. If the import occurs in the `__main__` module, correct path to the module cannot be built, resulting in an `ImportError`. Running the module with the `-m` flag processes the path of the `python` interpreter a bit differently.

---

A few following cells create a simple project structure that uses relative imports.

In [2]:
%%bash
rm -rf /tmp/relative_import
mkdir -p /tmp/relative_import/package

In [4]:
%%writefile /tmp/relative_import/package/module1.py
print("module1")

Writing /tmp/relative_import/package/module1.py


In [3]:
%%writefile /tmp/relative_import/package/intro_module.py
from . import module1

Writing /tmp/relative_import/package/intro_module.py


The following cell attempts to run `intro_module` as the main programm's main module.

In [6]:
!cd /tmp/relative_import/ && python3 package/intro_module.py

Traceback (most recent call last):
  File "/tmp/relative_import/package/intro_module.py", line 1, in <module>
    from . import module1
ImportError: attempted relative import with no known parent package


The result is the `ImportError`, which makes it clear that the error occurs due to relative import. The following cell runs `package.intro_module` with the `-m` flag. As a result, the programm executes without any problems.

In [8]:
!cd /tmp/relative_import/ && python3 -m package.intro_module

module1


**Note:** This won't work if the module you want to run doesn't have a parent package. The following cell demonstrates an attempt to run the `intro_module` from the `package` folder.

In [9]:
!cd /tmp/relative_import/package/ && python3 -m intro_module

Traceback (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/tmp/relative_import/package/intro_module.py", line 1, in <module>
    from . import module1
ImportError: attempted relative import with no known parent package
